In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import PIL
import time
import numpy as np
import os
import cv2

import hiddenlayer as hl

In [2]:
dataset=torchvision.datasets.ImageFolder(root=r"C:\Users\Peter King\python编程项目\python机器学习\Tello-Python-master\Tello_Video\test_photos")
print(dataset)
im_list=[]
target_list=[]
for i in dataset:
    im_list.append(np.array(i[0].resize((72,72),PIL.Image.BILINEAR)))
    target_list.append(i[1])
im_tensor=torch.Tensor(np.array(im_list)).transpose(1,3)
target_tensor=torch.Tensor(np.array(target_list))

Dataset ImageFolder
    Number of datapoints: 204
    Root location: C:\Users\Peter King\python编程项目\python机器学习\Tello-Python-master\Tello_Video\test_photos


In [11]:
#网络的定义
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(3,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)    #池化层r=36
        self.bn1 = nn.BatchNorm2d(64)      #batch内部归一化
        self.relu1 = nn.ReLU()             #激活函数

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)   #r=18
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.conv5 = nn.Conv2d(128,128, 3,padding=1)
        self.conv6 = nn.Conv2d(128, 128, 1)
        self.pool3 = nn.MaxPool2d(2, 2)   #r=9
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

        self.fc14 = nn.Linear(10368,256)  #16*128*
        self.drop1 = nn.Dropout2d()
        self.fc15 = nn.Linear(256,64)
        self.drop2 = nn.Dropout2d()
        self.fc16 = nn.Linear(64,2)


    def forward(self,x):
        #x = torch.tensor(x, dtype=torch.float32)  #这里必须注意！要改成float32形式的数据
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        #print(x.shape)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        #print(x.shape)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        #print(x.shape)

        x = x.view(-1,10368)
        #print(x.shape)
        x = F.relu(self.fc14(x))
        x = self.drop1(x)
        x = F.relu(self.fc15(x))
        x = self.drop2(x)
        x = self.fc16(x)

        return x

model=Net()

In [4]:
from sklearn.model_selection import StratifiedShuffleSplit

n_epochs=15
ss=StratifiedShuffleSplit(n_splits=n_epochs,test_size=0.25,train_size=0.75)
ge=ss.split(im_tensor,target_tensor)

In [5]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)
loss_function = nn.CrossEntropyLoss()
loss_record=[]
for epoch in range(n_epochs):
    train_index,test_index=next(ge)
    optimizer.zero_grad()
    output=model(im_tensor[train_index[:20]])
    loss=loss_function(output,target_tensor[train_index[:20]].long())
    print("train loss: %f"%loss.item())    
    loss_record.append(loss.item())
    loss.backward()
    optimizer.step()
    
    
    test_loss=loss_function(model(im_tensor[test_index]),target_tensor[test_index].long())
    print("test loss: %f"%test_loss.item())
    
    

In [22]:
output_list=[]
for i in range(10):
    output=model(im_tensor[i:i+16])
    for j in output:
        output_list.append(j.detach().numpy())